In [ ]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Modules

In [ ]:
import os
import warnings
import pickle
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import SubsetRandomSampler

import numpy as np
%matplotlib inline

import drive.MyDrive.GliomaSegmentation.my_bts.bts_dataset as dataset
import drive.MyDrive.GliomaSegmentation.my_bts.bts_model as model
import drive.MyDrive.GliomaSegmentation.my_bts.bts_classifier as classifier
import drive.MyDrive.GliomaSegmentation.my_bts.bts_plot as plot

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Computation Details')
# print(f'\tDevice Used: ({device})  {torch.cuda.get_device_name(torch.cuda.current_device())}\n')

print('Packages Used Versions:-')
print(f'\tPytorch Version: {torch.__version__}')

Computation Details
Packages Used Versions:-
	Pytorch Version: 1.13.0+cu116


## Initialize Hyperparameters

In [ ]:
# Dataset part used for testing
TEST_SPLIT = 0.2
# Batch size for training. Limited by GPU memory
BATCH_SIZE = 6
# mini_batch size for training 
MINI_BATCH = 64
# Dataset folder used
DATASET_USED = 'png_dataset'
# Full Dataset path
DATASET_PATH = "/content/drive/MyDrive/GliomaSegmentation/cheng_dataset"
# Training Epochs
EPOCHS = 100
# Filters used in UNet Model
FILTER_LIST = [16,32,64,128,256]
# Flag to train the model
TRAIN = True
# Flag to load saved model
LOAD_MODEL = False
# Flag to save model trained
SAVE_MODEL = True
# Model name to save or load.
MODEL_NAME = f"UNet-Model_{BATCH_SIZE}_batch_{MINI_BATCH}-cheng.pt"

print(f"Model Name :   {MODEL_NAME}")

Model Name :   UNet-Model_6_batch_64-cheng.pt


In [ ]:
def get_indices(length, new=True):
    """ Gets the Training & Testing data indices for a
    paticular "DATASET_USED".Stores the indices and returns
    them back when the same dataset is used.
    Parameters:
        length(int): Length of the dataset used.
        new(bool): Discard the saved indices and get new ones.
    Return:
        train_indices(list): Array of indices used for training purpose.
        test_indices(list): Array of indices used for testing purpose.
    """
    # Pickle file location of the indices.
    if not (os.path.isdir('dataset')):
        print(f'Creating dataset folder')
        os.mkdir('dataset')
    file_path = os.path.join('dataset',f'split_indices_{DATASET_USED}.p')
    data = dict()
    if os.path.isfile(file_path) and not new:
        # File found.
        with open(file_path,'rb') as file :
            data = pickle.load(file)
            return data['train_indices'], data['test_indices']
    else:
        # File not found or fresh copy is required.
        indices = list(range(length))
        np.random.shuffle(indices)
        split = int(np.floor(TEST_SPLIT * length))
        train_indices , test_indices = indices[split:], indices[:split]
        # Indices are saved with pickle.
        data['train_indices'] = train_indices
        data['test_indices'] = test_indices
        with open(file_path,'wb') as file:
            pickle.dump(data,file)
    return train_indices, test_indices


## Load Dataset

In [ ]:
tumor_dataset = dataset.TumorDataset(DATASET_PATH)
train_indices, test_indices = get_indices(3064)
train_sampler, test_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(test_indices)

trainloader = torch.utils.data.DataLoader(tumor_dataset, BATCH_SIZE, sampler=train_sampler)
testloader = torch.utils.data.DataLoader(tumor_dataset, 1, sampler=test_sampler)

## Load UNet Model

In [ ]:
unet_model = None
unet_classifier = None
if not LOAD_MODEL:
    # New model is created.
    unet_model = model.UNet(1, 1).to(device)
    unet_classifier = classifier.BTClassifier(unet_model,device)
else:
    # Saved model is loaded on memory.
    unet_model = model.UNet(1, 1)
    unet_classifier = classifier.BTClassifier(unet_model,device)
    unet_classifier.restore_model(os.path.join('/content/drive/MyDrive/GliomaSegmentation/',MODEL_NAME))
    print('Saved model loaded')

In [ ]:
# Training process
if TRAIN:
    unet_model.train()
    path = os.path.join('/content/drive/MyDrive/GliomaSegmentation/',MODEL_NAME) if SAVE_MODEL else None
    unet_train_history = unet_classifier.train(EPOCHS,trainloader,mini_batch=MINI_BATCH,save_best=path)
    print(f'Training Finished after {EPOCHS} epoches')

# Testing process on test data.
print(f'Testing in progress...')
unet_model.eval()
unet_score = unet_classifier.test(testloader)
print(f'\n\nDice Score {unet_score}')
# Dice Score 0.7446110107881675

Got Here!
Starting Training...


training:   0%|          | 0/100 [00:00<?, ?it/s]

    Batch: 64,	Batch Loss: 0.4196234
    Batch: 128,	Batch Loss: 0.4343171
    Batch: 192,	Batch Loss: 0.4617031
    Batch: 256,	Batch Loss: 0.4413905
    Batch: 320,	Batch Loss: 0.4580090
    Batch: 384,	Batch Loss: 0.4360389


training:   1%|          | 1/100 [02:06<3:28:33, 126.40s/it]

Epoch: 001,  Loss:0.4407105,  Time:126.36secs	Saved at loss: 0.4407105020
    Batch: 64,	Batch Loss: 0.4434686
    Batch: 128,	Batch Loss: 0.4425431
    Batch: 192,	Batch Loss: 0.4652929
    Batch: 256,	Batch Loss: 0.4681334
    Batch: 320,	Batch Loss: 0.4538482
    Batch: 384,	Batch Loss: 0.4324090


training:   2%|▏         | 2/100 [03:12<2:28:55, 91.18s/it] 

Epoch: 002,  Loss:0.4494872,  Time:66.53secs
    Batch: 64,	Batch Loss: 0.4452490
    Batch: 128,	Batch Loss: 0.4434475


training:   2%|▏         | 2/100 [03:41<3:01:13, 110.95s/it]


KeyboardInterrupt: ignored

In [ ]:
save_plot = f'{MODEL_NAME}-loss_graph.png'
plot.loss_graph(unet_train_history['train_loss'],save_plot)

NameError: ignored